### Imports

In [20]:
import pandas as pd

In [21]:
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

### Leemos y consolidamos los datasets

In [22]:
ventas = pd.read_csv('../Datasets/sell-in.txt', sep='\t')
productos = pd.read_csv('../Datasets/tb_productos.txt', sep='\t')
productos = productos.drop_duplicates()

condicion = ((productos['product_id'] == 20230) & (productos['cat3'] == 'SHAMPOO')) | \
            ((productos['product_id'] == 20623) & (productos['cat3'] == 'ACONDICIONADOR'))

productos = productos[~condicion]
stock = pd.read_csv('../Datasets/tb_stocks.txt', sep='\t')
productos_predecir = pd.read_csv('../Datasets/productos_a_predecir.csv')

ventas_grouped = ventas.groupby(['periodo', 'product_id']).agg({
    'plan_precios_cuidados': 'sum',
    'cust_request_qty': 'sum', # sirve de algo para el modelo? En zulip se me menciona que es la cantidad de solicitudes que hace el cliente, por lo que creo que no
    'cust_request_tn': 'sum',
    'tn': 'sum'
}).reset_index()

ventas_producto = pd.merge(ventas_grouped, productos, on=['product_id'], how='inner')
final_dataset = pd.merge(ventas_producto, stock, on=['product_id', 'periodo'], how='left')
final_dataset = pd.merge(final_dataset, productos_predecir, on=['product_id'], how='inner')

final_dataset.to_csv('../Datasets/final_dataset.csv', sep='\t', index=False)

In [23]:
final_dataset.isnull().sum()

periodo                      0
product_id                   0
plan_precios_cuidados        0
cust_request_qty             0
cust_request_tn              0
tn                           0
cat1                         0
cat2                         0
cat3                         0
brand                        0
sku_size                     0
stock_final              11622
dtype: int64

### Analisis exploratorio basico

#### Analisis inicial

In [24]:
final_dataset.shape

(22349, 12)

In [25]:
final_dataset.head(20)

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,20001,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
1,201702,20001,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
2,201703,20001,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
3,201704,20001,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
4,201705,20001,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
5,201706,20001,0,570,1575.82891,1520.06539,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
6,201707,20001,0,381,1086.47101,1030.67391,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
7,201708,20001,0,643,1289.66869,1267.39462,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
8,201709,20001,0,381,1356.96103,1316.94604,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
9,201710,20001,0,273,1441.60247,1439.75563,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN


In [26]:
final_dataset[final_dataset['cust_request_tn'] != final_dataset['tn']]

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,20001,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
1,201702,20001,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
2,201703,20001,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
3,201704,20001,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
4,201705,20001,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
22322,201910,20686,0,211,15.29067,14.99404,PC,CABELLO,SHAMPOO,NIVEA,400,39.84815
22324,201912,20686,0,168,9.89713,9.60049,PC,CABELLO,SHAMPOO,NIVEA,400,61.98351
22328,201912,20703,0,162,9.57806,9.46570,PC,CABELLO,Acond Mujer,NIVEA,400,0.38653
22342,201910,21087,0,56,0.97972,0.90802,PC,DEOS,Aero,NIVEA,65,4.12060


In [27]:
final_dataset.describe()

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,sku_size,stock_final
count,22349.000000,22349.000000,22349.000000,22349.000000,22349.000000,22349.000000,22349.000000,10727.000000
mean,201817.913687,20479.013423,1.249720,222.769744,51.364013,50.230669,447.639134,21.202147
std,81.433326,334.475371,12.981709,123.774437,129.425314,125.016406,831.235708,60.538364
min,201701.000000,20001.000000,0.000000,1.000000,0.000890,0.000890,1.000000,-13.666560
25%,201711.000000,20200.000000,0.000000,129.000000,3.010850,2.991610,89.000000,1.422785
50%,201809.000000,20411.000000,0.000000,207.000000,11.936620,11.900460,220.000000,6.120440
75%,201905.000000,20730.000000,0.000000,304.000000,36.272670,35.983630,475.000000,19.136220
max,201912.000000,21276.000000,289.000000,756.000000,2423.708810,2295.198320,10000.000000,1562.024480


In [28]:
final_dataset[(final_dataset['plan_precios_cuidados'] != 0) & (~final_dataset['plan_precios_cuidados'].isna())]

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
396,201701,20012,172,386,491.10893,476.39728,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
397,201702,20012,185,349,554.77790,509.04048,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
398,201703,20012,205,567,665.79620,649.49079,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
399,201704,20012,190,496,755.65972,749.91517,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
932,201909,20026,143,359,210.99220,209.49245,HC,ROPA LAVADO,Liquido,LIMPIEX,800,27.12045
...,...,...,...,...,...,...,...,...,...,...,...,...
21759,201908,20681,67,154,9.58617,9.58617,HC,HOGAR,Gel,MUSCULO,250,57.12376
21760,201909,20681,79,199,10.67498,10.67498,HC,HOGAR,Gel,MUSCULO,250,46.40773
21761,201910,20681,85,189,11.49681,11.49681,HC,HOGAR,Gel,MUSCULO,250,30.81752
21762,201911,20681,66,143,7.04877,7.04877,HC,HOGAR,Gel,MUSCULO,250,24.26914


#### Analizo si puede existir alguna venta de un producto en el mismo periodo que no sea plan_precios_cuidados y otras que si

In [29]:
ventas[(ventas['product_id'] == 20026) & (ventas['periodo'] == 201909) & (ventas['plan_precios_cuidados'] == 0)]

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn


In [30]:
def has_both_types(group):
    has_cuidados = (group['plan_precios_cuidados'] != 0).any()
    has_no_cuidados = (group['plan_precios_cuidados'] == 0).any()
    # display(group)
    return has_cuidados and has_no_cuidados

filtered_groups = ventas.groupby(['product_id', 'periodo']).filter(has_both_types)
filtered_groups

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn


Como observo que no existe ningun producto, en ningun periodo, que fuera a su vez precio_cuidoda y que no lo fuera, procedo a modificar esa variable en dataset_final para transformarla en boolean en vez de en suma:

In [31]:
final_dataset['plan_precios_cuidados'] = final_dataset['plan_precios_cuidados'].apply(lambda x: 1 if x > 0 else 0)

In [32]:
final_dataset[(final_dataset['plan_precios_cuidados'] != 0) & (~final_dataset['plan_precios_cuidados'].isna())]

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
396,201701,20012,1,386,491.10893,476.39728,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
397,201702,20012,1,349,554.77790,509.04048,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
398,201703,20012,1,567,665.79620,649.49079,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
399,201704,20012,1,496,755.65972,749.91517,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
932,201909,20026,1,359,210.99220,209.49245,HC,ROPA LAVADO,Liquido,LIMPIEX,800,27.12045
...,...,...,...,...,...,...,...,...,...,...,...,...
21759,201908,20681,1,154,9.58617,9.58617,HC,HOGAR,Gel,MUSCULO,250,57.12376
21760,201909,20681,1,199,10.67498,10.67498,HC,HOGAR,Gel,MUSCULO,250,46.40773
21761,201910,20681,1,189,11.49681,11.49681,HC,HOGAR,Gel,MUSCULO,250,30.81752
21762,201911,20681,1,143,7.04877,7.04877,HC,HOGAR,Gel,MUSCULO,250,24.26914


In [33]:
# Observo que tambien los Na de plan_precios_cuidados se mapearon a 0
final_dataset[final_dataset['plan_precios_cuidados'].isna()]

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final


#### Defino la variable target

In [34]:
final_dataset.rename(columns={'tn': 'y'}, inplace=True)
final_dataset.to_csv('../Datasets/final_dataset.csv', sep='\t', index=False)
final_dataset

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,20001,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
1,201702,20001,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
2,201703,20001,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
3,201704,20001,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
4,201705,20001,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
22344,201912,21087,0,115,1.05790,1.02205,PC,DEOS,Aero,NIVEA,65,5.79239
22345,201909,21214,0,3,0.34250,0.34250,PC,DEOS,RollOn,NIVEA,50,2.59152
22346,201910,21214,0,46,0.21735,0.21735,PC,DEOS,RollOn,NIVEA,50,1.65982
22347,201911,21214,0,88,0.84012,0.84012,PC,DEOS,RollOn,NIVEA,50,0.52214


#### Analisis nulos

In [35]:
# TODO
final_dataset.isnull().sum()

periodo                      0
product_id                   0
plan_precios_cuidados        0
cust_request_qty             0
cust_request_tn              0
y                            0
cat1                         0
cat2                         0
cat3                         0
brand                        0
sku_size                     0
stock_final              11622
dtype: int64

El stock no parece ser util para los modelos, y es a su vez, la unica variable con valores nulos, por lo que no haremos ninguna tratamiento de momento.

#### Feature Engineering

Agregamos una columna que indica si el mes es cierre de Q, y otra columna que indique la "edad" del producto, considerando la fecha de la primer venta en el dataset

In [36]:
final_dataset['periodo_temp'] = final_dataset['periodo'].astype(str)
final_dataset['close_quarter'] = final_dataset['periodo_temp'].str[-2:].isin(['03', '06', '09', '12']).astype(int)

final_dataset['first_period'] = final_dataset.groupby('product_id')['periodo'].transform('min')
final_dataset['first_period'] = final_dataset['first_period'].astype(str)

final_dataset['age'] = ((final_dataset['periodo_temp'].str[:4].astype(int) - final_dataset['first_period'].str[:4].astype(int)) * 12 +
                        (final_dataset['periodo_temp'].str[-2:].astype(int) - final_dataset['first_period'].str[-2:].astype(int)))


final_dataset.drop(columns=['periodo_temp', 'first_period'], inplace=True)
display(final_dataset.head())

final_dataset.to_csv('../Datasets/final_dataset.csv', sep='\t', index=False)

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age
0,201701,20001,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
1,201702,20001,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,1
2,201703,20001,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,1,2
3,201704,20001,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,3
4,201705,20001,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,4


#### Generacion de file para kaggle (para usar en los modelos)

In [37]:
#generar dataset Kaggle
#df.to_csv('../Datasets/df.csv', sep=',', index=False)

#### Formula para calcular el error (para usar en los modelos)

In [38]:
# df_mm['AbsoluteError'] = abs(df_mm['y'] - df_mm['prediction'])
# total_forecast_error = df_mm['AbsoluteError'].sum() / df_mm['y'].sum()